# 로지스틱 회귀 - 타이타닉 생존률 예측

## 로지스틱 회귀 수행

### 1) 패키지 준비

In [ ]:
import sys
sys.path.append('../../')
import helper

import numpy as np
import seaborn as sb
from pandas import read_excel, DataFrame, get_dummies
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential # 순서층을 구성하는 모델 객체 생성 기능
from tensorflow.keras.layers import Dense      # 모델 객체에 학습층을 쌓기 위한 클래스
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### 2) 데이터셋 준비

In [ ]:
origin = read_excel('https://data.hossam.kr/E05/titanix.xlsx')
origin.info()

### 3) 데이터 전처리
- 결측치 확인

In [ ]:
origin.isna().sum()

- 결측치 정제
    - 객실번호 (Cabin) - 선실이 부여되지 않은 경우 객실 등급에 따른 차이가 있는지 여부를 확인

In [ ]:
df_tmp = origin.filter(['Pclass', 'Cabin']).query('Cabin.isnull()')
df_tmp.fillna(0, inplace=True)
df_tmp.groupby('Pclass').count()

> 객실 번호는 생존 여부에 영향이 없을 것으로 판단하고 변수 자체를 제거 (열단위 제거)

In [ ]:
df1 = origin.drop('Cabin', axis=1)
df1.head()

- 결측치 정제
    - 탑승지 (Embarked) - 탑승지 데이터가 결측치인 경우는 2건 밖에 되지 않기 때문에 데이터 정제 과정에서 제거 (행단위)

In [ ]:
df2 = df1.query('Embarked.notnull()')
df2.head()

- 결측치 정제
    - 나이 (age)
        - 탑승객의 생존 여부에 중요한 영향을 미치는 요인이라고 짐작하고 삭제하지 않기로 결정
        - `연속형 데이터의 결측치를 대체`하기에 가장 적합한 값은 `중앙값 (median)`이므로 중앙값으로 대체

In [ ]:
df3 = df2.copy()
df3['Age'].fillna(df3['Age'].median(), inplace=True)
df3.isna().sum()

- 불필요한 필드 제거
    - 탑승객 번호 - 단순한 일련번호이므로 생존률에 영향을 주지 않는 값이라고 판단하고 이 값을 인덱스로 설정

In [ ]:
df4 = df3.set_index('PassengerId')
df2.head()

- 불필요한 필드 제거
    - 이름과 티켓번호 - 생존 여부에 영향을 주지 않는다고 판단하고 제거

In [ ]:
df5 = df4.drop(['Name', 'Ticket'], axis=1)
df5.head()

- 더미변수 처리

In [ ]:
cda_df = get_dummies(df5, columns=['Pclass', 'Sex', 'Embarked'],
                     drop_first=True,
                     dtype='int')
cda_df.head()

### 4) 탐색적 데이터 분석
> E05의 연습문제 2번으로 진행한 바 있으므로 생략

### 5) 데이터셋 분할하기
- 랜덤시드 고정

In [ ]:
np.random.seed(777)

- 훈련데이터(독립변수)와 레이블(종속변수) 구분하기

In [ ]:
x = cda_df.drop('Survived', axis=1)
x.head()

y = cda_df[['Survived']]
y.head()

- 데이터 표준화
> 일단 생략해보기로 함

- 훈련데이터와 검증데이터로 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 6) 모델 개발
- 모델 정의

In [ ]:
my_model = Sequential
my_model.add(Dense(32, activation='relu', input_shape=(9,)))
my_model.add(Dense(1, activation='sigmoid'))
my_model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])
my_model.summary()

- 학습하기

In [ ]:
result = my_model.fit(x_train, 
                      y_train, 
                      epochs=500, 
                      validation_data=(x_test, y_test)), 
                      callbacks = [EarlyStopping(monitor='val_loss',
                                                 patience=5,
                                                 verbose=1),
                                   ReduceLROnPlateau(monitor='val_loss',
                                                     patience=3,
                                                     factor=0.5,
                                                     min_lr=0.0001,
                                                     verbose=1)

### 7) 학습 결과 평가
- 기본적인 머신러닝 학습률 평가

In [ ]:
helper.tf_result_plot(result)

evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실률: %f, 최종 훈련 정확도: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test,y_test)
print('최종 검증 손실률: %f, 최종 검증 정확도: %f' % (evaluate2[0], evaluate2[1]))

- 로지스틱 회귀 결과 검정
> helper 기능 추가

In [ ]:
logit_result = helper.tf_logit_result(my_model,
                                      result,
                                      x_train,
                                      y_train)

In [2]:
logit_result.cmdf

NameError: name 'logit_Result' is not defined

In [ ]:
logit_result.odds_rate_df

### 8) 학습 결과 적용
- 실제 데이터로 예측해보기
- 예측을 위한 데이터프레임 준비

In [ ]:
test_df = DataFrame({
    'dicaprio': [19, 0, 0, 0, 0, 1, 1, 0, 0],  # 영화속 남자 주인공 데이터
    'winslet': [17, 1, 2, 100, 0, 0, 0, 1, 0], # 영화속 여자 주인공 데이터
    'someone': [41, 1, 0, 50, 1, 0, 1, 0, 0]   # 임의의 데이터
}, index=['Age','SibSp','Parch','Fare','Pclass_2','Pclass_3','Sex_male','Embarked_Q','Embarked_S'])
test_df

- 결과 예측하기

In [ ]:
results = my_model.predict(test_df.T)
data_count, case_count = results.shape
print("%d개의 임의의 데이터가 %d개의 경우의 수를 갖는다." % (data_count, case_count))
print(results)

In [ ]:
for i,v in enumerate(results.flatten()):
    print('%s의 생존 확률은 %0.1f%% 입니다.' % (test_df.columns[i], v*100))